In [1]:
using LinearAlgebra                                                             
using Test                                                                                       
using Distributions                                                             
using DataFrames                                                                          
using CSV                                                                              
using Polynomials  
                                                                                 
include("../computation/finitedifference.jl")                                                   
include("../quadrature/quadrature.jl")                                             
include("../transforms/transforms.jl")                                             
include("../priors/priors.jl")                                                     
include("../bayesopt/incremental.jl")                                              
include("../kernels/kernel.jl")                                                    
include("../datastructs.jl")                                                       
include("../computation/buffers0.jl") #datastruct, kernel, incremental, quadrature 
include("../model0.jl") #buffers, datastructs, several auxiliary                   
include("../computation/tdist.jl") #model0 and buffer0  

LoadError: syntax: extra token ")" after end of expression

In [2]:
function covariate_fun(x, p)
    n = size(x, 1)
    d = size(x, 2)
    if p == 1
        return ones(n, 1)
    elseif p == 1 + d
        return hcat(ones(n), x)
    else
         throw(ArgumentError("Only support constant or linear convariate."))
    end
end

covariate_fun (generic function with 1 method)

In [3]:
# Data and setup for different tests

# choice of covariate functions
p = 1
p = 3

# training
x1 = 4 .* rand(nx, 2) .- 2 
x2 = [-0.28340   0.62098;
     -1.59902   1.86275;
      0.49347  -1.46311;
      1.50624  -1.42874; 
      1.57966  -1.4987;  
     -1.35577   0.33419;
      1.57967   0.4927;  
      1.30226   1.30865; 
      1.84554  -1.71706; 
      1.45705   0.264961]
x3 = [-0.801151   -0.543169;   
      -0.94552     1.09677;    
       0.947512   -0.827147;   
      -1.42516     0.916257;   
      -1.35157     0.449443;   
       0.266677    1.31536;    
      -0.555493    0.841922;   
      -0.0235545   0.000649033;
      -0.339027    0.843967;   
      -1.7816      1.52251; ]

# testing
x01 = 4 .*rand(1, 2) .- 2
x02 = [-1.0097 0.3929]
x03 = [-0.936599  -1.94613]

# underlying 2d function 
test_fun1(x, y) = x^2 * exp(- x^2 - y^2) 


UndefVarError: UndefVarError: nx not defined

# Summary
Settings parameters: \
p: choices of covariates, p = 1 if constant and p = 2 if linear \
x: training data \
x0: testing data

Tests:
1. p=3 or p=1, x=x2, x0=x02 \
   pdf blows up near 0, numerical issue (g'(y0) blows up when y0~=0?)
2. p=1, x=x3, x0=x03 \
    pdf weird shape (no peak, uniformish)
3. p=3, x=x3, x0=x03 \
    pdf blows up near 0, and pdf support ~= $[0,5]$

In [4]:
# parameter setting: change for different tests

p = 1 # choose covariate basis function, only support 1 or 1+dim(x)
x = x2 # choose training data 
x0 = x02 # choose testing data
test_fun = test_fun1 # choose underlying function

#####################################

# setup values and covariates (no changes needed)
Fx = covariate_fun(x, p)
y = [test_fun(_x[1], _x[2]) for _x in zip(x[:,1], x[:,2])]
ymax = maximum(y)
y /= ymax
trainingData1 = trainingData(x, Fx, y) 

Fx0 = covariate_fun(x0, p)
y0_true = test_fun(x0[1], x0[2])/ymax
println("Prediction at x0 = $x0")
println("Covariates at x0: $Fx0")
println("Should expect peak near $y0_true")


UndefVarError: UndefVarError: x3 not defined

In [5]:
d = getDimension(trainingData1)
n = getNumPts(trainingData1)
p = getCovDimension(trainingData1)
rangeθ = [0.25 150]
rangeλ = [0.5 5]

UndefVarError: UndefVarError: trainingData1 not defined

In [6]:
btg1 = btg(trainingData1, rangeθ, rangeλ);
(dpdf, pdf, cdf) = solve(btg1);
dpdf_fixed = y0 -> dpdf(x0, Fx0, y0) 
pdf_fixed = y0 -> pdf(x0, Fx0, y0)
cdf_fixed = y0 -> cdf(x0, Fx0, y0)

UndefVarError: UndefVarError: trainingData1 not defined

In [7]:
plt(pdf_fixed, 1e-5, 5, 100)
plt!(cdf_fixed, 1e-5, 5, 100, title = "PDF and CDF of Bayesian Predictive Distribution")

UndefVarError: UndefVarError: pdf_fixed not defined